In [110]:
import pandas as pd
import requests
import io

In [111]:
API_KEY = "861E6E6E-6BD2-3C50-8731-82E4EC90A5E3"


In [112]:
# API Documentation at https://quickstats.nass.usda.gov/api/

params = {
    "source_desc": "SURVEY",
    "sector_desc": "CROPS",
    "group_desc": "FIELD CROPS",
    "commodity_desc": "CORN",
    "statisticcat_desc": "YIELD",
    "short_desc": "CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",
    "domain_desc": "TOTAL",
    "agg_level_desc": "COUNTY",
    "year__GE": "2013"
}

query = "&".join([f"{key}={val}" for key, val in params.items()])

URL = f"https://quickstats.nass.usda.gov/api/api_GET/?key={API_KEY}&{query}&format=CSV"
headers = {'User-Agent': ''} # NEEDED BECAUSE USDA BLOCKS 'non-browser' traffic


In [169]:
response = requests.get(URL,headers = {'User-Agent': ''})

In [170]:
df = pd.read_csv(io.StringIO(response.content.decode("UTF-8")), thousands=",")

In [171]:
print(len(df))
print(df.columns)

14524
Index(['source_desc', 'sector_desc', 'group_desc', 'commodity_desc',
       'class_desc', 'prodn_practice_desc', 'util_practice_desc',
       'statisticcat_desc', 'unit_desc', 'short_desc', 'domain_desc',
       'domaincat_desc', 'agg_level_desc', 'state_ansi', 'state_fips_code',
       'state_alpha', 'state_name', 'asd_code', 'asd_desc', 'county_ansi',
       'county_code', 'county_name', 'region_desc', 'zip_5', 'watershed_code',
       'watershed_desc', 'congr_district_code', 'country_code', 'country_name',
       'location_desc', 'year', 'freq_desc', 'begin_code', 'end_code',
       'reference_period_desc', 'week_ending', 'load_time', 'Value', 'CV (%)'],
      dtype='object')


That's a lot of columns. Let's keep the ones we care about.

In [116]:
columnsToKeep = ["year", "state_name", 'county_name', "Value"]
df = df[columnsToKeep]

In [117]:
df.head()

,year,state_name,county_name,Value
0,2019,ALABAMA,OTHER (COMBINED) COUNTIES,920000
1,2018,ALABAMA,OTHER (COMBINED) COUNTIES,322000
2,2017,ALABAMA,OTHER (COMBINED) COUNTIES,254000
3,2016,ALABAMA,OTHER (COMBINED) COUNTIES,192000
4,2015,ALABAMA,OTHER (COMBINED) COUNTIES,990000


Now that we have all the corn production data by county for all of the states,
let's further prune this for the counties in the state's that we want.

In [118]:
corn_belt_states = set([
    "MINNESOTA",
    "SOUTH DAKOTA",
    "NEBRASKA",
    "KANSAS",
    "IOWA",
    "WISCONSIN",
    "ILLINOIS",
    "MISSOURI",
    "INDIANA",
    "OHIO"
])

df_corn = df[df["state_name"].isin(set(corn_belt_states))].reset_index(drop=True)

In [126]:
print(df_corn['state_name'].unique())
print(len(df_corn['county_name'].unique()))
print(len(df_corn['year'].unique()))

['ILLINOIS' 'INDIANA' 'IOWA' 'KANSAS' 'MINNESOTA' 'MISSOURI' 'NEBRASKA'
 'OHIO' 'SOUTH DAKOTA' 'WISCONSIN']
568
9


In [151]:
df_corn.groupby(by=['county_name']).agg({'county_name': 'count'}).rename(columns={"county_name": "count"}).reset_index().sort_values("count", ascending=False)

,county_name,count
372,OTHER (COMBINED) COUNTIES,349
236,JEFFERSON,67
536,WASHINGTON,67
233,JACKSON,67
83,CLAY,61
...,...,...
442,RUSSELL,1
480,STANLEY,1
494,SUMMIT,1
263,KOOCHICHING,1


There appears to be some duplicate counties going on.

Other (COMBINED) Counties make sense if multiple states describe extraneous counties with the same name.

However, the same case shouldn't exist for other counties, unless any given 2 states simply share a county name despite
these 2 counties actually being different.

In [153]:
# Validate these "duplicate" counties

df_corn[df_corn["county_name"] == "CLAY"]['state_name'].unique()

array(['ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'MINNESOTA', 'MISSOURI',
       'NEBRASKA', 'SOUTH DAKOTA'], dtype=object)

That's impressive. By a simple google search, it's pretty easy to validate that "Clay" county is a distinct county in all of these states.


Now that that's out of the way, we can proceed with trust in our data.

Currently, these values are in terms of bushels. Let's convert it to (1000 metric tons).

In [120]:
df_corn["Value"] = df_corn["Value"]/39.3679/1000

In [163]:
df_dist = df_corn.groupby('state_name').agg(count=('Value', 'count'), total=('Value', 'sum')).reset_index()
df_dist

,state_name,count,total
0,ILLINOIS,841,491402.894236
1,INDIANA,758,218773.162907
2,IOWA,859,565891.246422
3,KANSAS,685,152295.652041
4,MINNESOTA,657,313846.738078
5,MISSOURI,641,118461.233645
6,NEBRASKA,704,393666.159485
7,OHIO,733,129338.115571
8,SOUTH DAKOTA,453,170482.956927
9,WISCONSIN,556,115374.200808


In [164]:
df_dist['percentage'] = round(df_dist['total']/sum(df_dist['total']), 2)
df_dist

,state_name,count,total,percentage
0,ILLINOIS,841,491402.894236,0.18
1,INDIANA,758,218773.162907,0.08
2,IOWA,859,565891.246422,0.21
3,KANSAS,685,152295.652041,0.06
4,MINNESOTA,657,313846.738078,0.12
5,MISSOURI,641,118461.233645,0.04
6,NEBRASKA,704,393666.159485,0.15
7,OHIO,733,129338.115571,0.05
8,SOUTH DAKOTA,453,170482.956927,0.06
9,WISCONSIN,556,115374.200808,0.04


In [166]:
df_corn["Value"].describe()

count     6887.000000
mean       387.619045
std        446.986319
min          0.254014
25%        120.428064
50%        299.457172
75%        563.987411
max      19878.454274
Name: Value, dtype: float64

It's clear that we have a pretty representative data set of corn production for all the states in the United States corn belt.

Let's save the dataset as a csv and get the rest of our data.

In [168]:
df_corn.to_csv("../data/crop_data.csv", index=False)